# Querying MPC for objects in Allen Fields

## First load the pointing group info

In [76]:
import numpy as np
import pickle
from astropy.io.votable import parse
import matplotlib.pyplot as plt
import pandas as pd

import healpy as hp
import pointing_groups as pg

import requests
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.time import Time

from astropy.io import fits
from astropy.wcs import WCS
from astropy.wcs import NoConvergence

%matplotlib inline

In [2]:
with open('PickledPointings.pkl', 'rb') as f:
    Pointing_Groups = pickle.load(f)
    
den_visit,den_ccd,den_stellarnum = np.loadtxt('ref_data.txt',unpack=True)
unique_den_visit = np.unique(den_visit)
max_stellarnum = []
_= [ max_stellarnum.append(np.max(den_stellarnum[den_visit==visit])) for visit in unique_den_visit ]
max_stellarnum = np.array(max_stellarnum)

for i,group in enumerate(Pointing_Groups):
    group_max = np.concatenate([ (max_stellarnum[(unique_den_visit==x)]) for x in np.array(Pointing_Groups[i]["visit_id"],dtype=np.float_) ])
    if not(group_max.size):
        group_max=[-9999.]
    group_max = np.max(group_max)
    Pointing_Groups[i]['stellar_density'] = group_max
    
# visitToIndex is a dictionary that returns the index of a given visit
visitToIndex = {}
for i,pointing in enumerate(Pointing_Groups):
    for visit in pointing['visit_id']:
        visitToIndex[str(visit)] = i

In [4]:
Pointing_Groups[0].head()

,visit_id,date_obs,ra,dec,filename,survey_night,stellar_density
0,439116,b'2015-05-20 10:12:35.083',257.655417,-29.255417,b'c4d_150520_101400_ooi_VR_v4.fits.fz',20,40964.0
1,439120,b'2015-05-20 10:17:14.409',257.655333,-29.256250,b'c4d_150520_101841_ooi_VR_v4.fits.fz',20,40964.0
2,439124,b'2015-05-20 10:23:13.327',257.657042,-29.256778,b'c4d_150520_102438_ooi_VR_v4.fits.fz',20,40964.0
3,439128,b'2015-05-20 10:29:16.399',257.656083,-29.255667,b'c4d_150520_103042_ooi_VR_v4.fits.fz',20,40964.0
4,439707,b'2015-05-21 10:02:26.888',257.652042,-29.258389,b'c4d_150521_100355_ooi_VR_v2.fits.fz',21,40964.0


## Set up tools for MPC querying

In [52]:
url     = 'https://www.minorplanetcenter.net/cgi-bin/mpcheck.cgi'
headers = {}

payload = {'year':'2018',
           'month':'01',
           'day': '16.67',
           'which':'obs',
           'ra':'',
           'decl':'',
           'TextArea':'',
           'radius':'90',
           'limit':'30.0',
           'oc':'500',
           'sort':'d',
           'mot':'h',
           'tmot':'s',
           'pdes':'u',
           'needed':'f',
           'ps':'n',
           'type':'1'
          }

In [53]:
def create_obs_table(df):
    """
    Create the input to the MPC post request.
    
    Input
    -----
    df : pandas dataframe
    The dataframe of observations for a particular request.
    
    Output
    ------
    textarea : string
    The string that should be the 'textarea' value in the payload dictionary
    """
    
    textarea = ''
    
    for idx in range(len(df)):
        c = SkyCoord(df['ra'].iloc[idx]*u.degree, df['dec'].iloc[idx]*u.degree)
        ra_hms = c.ra.hms
        dec_dms = c.dec.dms
        date_obs = df['date_obs'].iloc[idx].decode()#[2:-1]
        time_obj = Time(date_obs, format='iso', scale='utc')
        if dec_dms.d != 0:
            name = ("     %07i   %s %s %s.%s %02i %02i %06.3f%+03i %02i %05.2f                     W84\n" %
                        (df['visit_id'].iloc[idx], date_obs[:4], date_obs[5:7], 
                         date_obs[8:10], str(time_obj.mjd)[6:11],
                         ra_hms.h, ra_hms.m, ra_hms.s,
                         dec_dms.d, np.abs(dec_dms.m), np.abs(dec_dms.s)))
        else:
            if copysign(1, dec_dms.d) == -1.0:
                dec_dms_d = '-00'
            else:
                dec_dms_d = '+00'
            name = ("     %07i   %s %s %s.%s %02i %02i %06.3f%s %02i %05.2f                     W84\n" %
                        (df['visit_id'].iloc[idx], date_obs[:4], date_obs[5:7],
                         date_obs[8:10], str(time_obj.mjd)[6:11],
                         ra_hms.h, ra_hms.m, ra_hms.s,
                         dec_dms_d, np.abs(dec_dms.m), np.abs(dec_dms.s)))
        textarea += name
        
    return textarea

In [54]:
def parse_results(result):
    
    results_cut = result.text.split('\n')[12:-49]
    
    visit_id = []
    name = []
    ra_hour = []
    ra_min = []
    ra_sec = []
    dec_deg = []
    dec_min = []
    dec_sec = []
    v_mag = []
    ra_motion = []
    dec_motion = []
    
    for line in results_cut:
        #print(line)
        visit_id.append(int(line[6:12]))
        name.append(line[12:36])
        ra_hour.append(int(line[38:40]))
        ra_min.append(int(line[41:43]))
        ra_sec.append(float(line[44:48]))
        dec_deg.append(int(line[49:52]))
        dec_min.append(int(line[53:55]))
        dec_sec.append(int(line[56:58]))
        try:
            v_mag.append(float(line[60:64]))
        except ValueError:
            v_mag.append(-99.0)
        ra_motion.append('%s%i' % (line[84], int(line[82:84])))
        dec_motion.append('%s%i' % (line[91], int(line[89:91])))
            
    results_df = pd.DataFrame(np.array([visit_id, name, ra_hour, ra_min, ra_sec, 
                                       dec_deg, dec_min, dec_sec, v_mag, 
                                       ra_motion, dec_motion]).T, 
                             columns=['visit_id', 'name', 'ra_hour', 'ra_min', 'ra_sec', 
                                       'dec_deg', 'dec_min', 'dec_sec', 'v_mag', 
                                       'ra_motion', 'dec_motion'])
    
    results_df['visit_id'] = pd.to_numeric(results_df['visit_id'])
    results_df['ra_hour'] = pd.to_numeric(results_df['ra_hour'])
    results_df['ra_min'] = pd.to_numeric(results_df['ra_min'])
    results_df['ra_sec'] = pd.to_numeric(results_df['ra_sec'])
    results_df['dec_deg'] = pd.to_numeric(results_df['dec_deg'])
    results_df['dec_min'] = pd.to_numeric(results_df['dec_min'])
    results_df['dec_sec'] = pd.to_numeric(results_df['dec_sec'])
    results_df['v_mag'] = pd.to_numeric(results_df['v_mag'])
    results_df['ra_motion'] = pd.to_numeric(results_df['ra_motion'])
    results_df['dec_motion'] = pd.to_numeric(results_df['dec_motion'])
    
    return results_df

In [55]:
def run_mpc_requests(dataframe, field_label):
    
    """
    Inputs
    ------
    dataframe: pandas dataframe, pointing group dataframe
    
    field_label: str, identifier for field
    
    Returns
    -------
    results_df: pandas dataframe with MPC results for field
    """
    
    url     = 'https://www.minorplanetcenter.net/cgi-bin/mpcheck.cgi'
    headers = {}

    payload = {'year':'2018',
               'month':'01',
               'day': '16.67',
               'which':'obs',
               'ra':'',
               'decl':'',
               'TextArea':'',
               'radius':'90',
               'limit':'30.0',
               'oc':'500',
               'sort':'d',
               'mot':'h',
               'tmot':'s',
               'pdes':'u',
               'needed':'f',
               'ps':'n',
               'type':'1'
              }
    
    results_df = None

    ta = create_obs_table(dataframe)
    payload['TextArea'] = ta
    print(ta)
    res = requests.post(url, data=payload, headers=headers)

    if results_df is None:
        results_df = parse_results(res)
        results_df['field'] = field_label
    else:
        label_results_df = parse_results(res)
        label_results_df['field'] = field_label
        results_df = results_df.append(label_results_df, ignore_index=True)
            
    return results_df

## Pick a pointing group and get data with `run_mpc_requests`

This will query the online MPC database and get a dataframe with all MPC objects within 90 arcminutes of the center of each pointing.

In [67]:
pg_num = 300

In [68]:
results_df = run_mpc_requests(Pointing_Groups[pg_num], str(pg_num))

     0433932   2015 04 18.27432 13 22 27.430-08 21 51.00                     W84
     0433936   2015 04 18.27756 13 22 27.460-08 21 49.60                     W84
     0433940   2015 04 18.28078 13 22 27.500-08 21 52.30                     W84
     0433945   2015 04 18.28872 13 22 27.620-08 21 51.80                     W84
     0433949   2015 04 18.29191 13 22 27.610-08 21 51.60                     W84
     0434610   2015 04 19.26575 13 22 27.680-08 21 50.10                     W84
     0434614   2015 04 19.26900 13 22 27.750-08 21 49.90                     W84
     0434618   2015 04 19.27225 13 22 27.670-08 21 54.60                     W84
     0434622   2015 04 19.27547 13 22 27.920-08 21 51.60                     W84
     0434626   2015 04 19.27871 13 22 27.690-08 21 52.20                     W84
     0435498   2015 04 21.26670 13 22 27.340-08 22 00.00                     W84
     0435502   2015 04 21.27007 13 22 27.200-08 22 04.50                     W84
     0435506   2015 04 21.27

In [69]:
results_df

,visit_id,name,ra_hour,ra_min,ra_sec,dec_deg,dec_min,dec_sec,v_mag,ra_motion,dec_motion,field
0,433932,2015 GF54,13,22,35.2,-8,24,57,24.6,-2,1,300
1,433932,(68766) 2002 EN102,13,22,8.9,-8,19,8,19.4,-17,10,300
2,433932,(390300) 2013 AS60,13,22,16.9,-8,29,18,20.6,-19,8,300
3,433932,2015 GU55,13,21,54.3,-8,19,41,23.1,-3,1,300
4,433932,2005 SL205,13,22,4.5,-8,28,47,22.0,-35,23,300
5,433932,2015 FF293,13,22,30.5,-8,11,7,20.7,-27,19,300
6,433932,(88731) 2001 SJ36,13,21,42.0,-8,25,34,18.6,-31,5,300
7,433932,(14791) Atreus,13,22,57.9,-8,10,40,19.3,-17,7,300
8,433932,(435713) 2008 UF26,13,23,22.9,-8,21,47,19.2,-38,0,300
9,433932,2005 SJ137,13,21,36.2,-8,27,21,22.1,-36,12,300


## Get pixel values and CCD Number

In [70]:
results_df['ccd'] = -99
results_df['x_pixel'] = -99
results_df['y_pixel'] = -99

In [71]:
results_df['total_motion'] = np.sqrt(results_df['ra_motion']**2. + results_df['dec_motion']**2.)

In [72]:
# Can comment this out if you don't want to select on slow movers (total_motion is arcsec/hr.)
res_df = results_df.query('total_motion < 10.')
res_df = res_df.reset_index(drop=True)

In [73]:
res_df.head(10)

,visit_id,name,ra_hour,ra_min,ra_sec,dec_deg,dec_min,dec_sec,v_mag,ra_motion,dec_motion,field,ccd,x_pixel,y_pixel,total_motion
0,433932,2015 GF54,13,22,35.2,-8,24,57,24.6,-2,1,300,-99,-99,-99,2.236068
1,433932,2015 GU55,13,21,54.3,-8,19,41,23.1,-3,1,300,-99,-99,-99,3.162278
2,433932,2015 GY53,13,22,9.7,-8,2,44,24.3,-8,5,300,-99,-99,-99,9.433981
3,433932,2015 GH56,13,23,34.5,-8,3,11,24.3,-2,1,300,-99,-99,-99,2.236068
4,433932,2015 GS56,13,24,22.7,-8,14,18,24.3,-2,1,300,-99,-99,-99,2.236068
5,433932,2015 GG56,13,20,45.5,-8,3,30,24.1,-2,1,300,-99,-99,-99,2.236068
6,433932,2015 GF56,13,20,23.4,-8,15,7,24.4,-2,1,300,-99,-99,-99,2.236068
7,433932,2015 GQ56,13,25,13.0,-8,18,8,23.9,-2,0,300,-99,-99,-99,2.000000
8,433932,2015 GO56,13,25,20.7,-8,25,30,24.4,-2,0,300,-99,-99,-99,2.000000
9,433932,2015 GT56,13,25,37.1,-8,10,44,24.5,-2,0,300,-99,-99,-99,2.000000


In [74]:
data_loc = '/astro/store/epyc/users/smotherh/DECAM_Data_Reduction/warps/%i' % pg_num

In [75]:
night_visits = np.unique(res_df['visit_id'])

In [87]:
for visit in night_visits:
    df_visit = res_df.query('visit_id == %i' % visit)
    print(data_loc)
    print(visit)
    for i in range(1, 63):
        #print(i)
        try:
            #print('%s/v%i/diffexp-%02i.fits' % (data_loc, visit, i))
            hdulist = fits.open('%s/%i-%02i.fits' % (data_loc, visit, i))
            w = WCS(hdulist[1].header)
        except FileNotFoundError:
            print(visit, i)
            continue
        for idx, obj_row in df_visit.iterrows():
            ra = '%i:%i:%.1f' % (obj_row['ra_hour'], obj_row['ra_min'], obj_row['ra_sec'])
            dec = '%i:%i:%.1f' % (obj_row['dec_deg'], obj_row['dec_min'], obj_row['dec_sec'])
            c = SkyCoord(ra, dec, frame='icrs', unit=(u.hourangle, u.deg))
            try:
                x_pix, y_pix = c.to_pixel(w)
                if (x_pix < 2010) and (x_pix > 0):
                    if (y_pix < 4100) and (y_pix > 0):
                        print(obj_row['name'], ra, dec)
                        print(x_pix, y_pix)
                        res_df['x_pixel'].iloc[idx] = x_pix
                        res_df['y_pixel'].iloc[idx] = y_pix
                        res_df['ccd'].iloc[idx] = i
            except NoConvergence:
                continue

/astro/store/epyc/users/smotherh/DECAM_Data_Reduction/warps/300
433932
433932 2
          2015 GK56      13:20:5.7 -7:39:9.0
1427.0095524054123 394.8008499887256


/epyc/opt/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


          2015 GG56      13:20:45.5 -8:3:30.0
230.27144927823076 2670.4870408432485
          2015 GY53      13:22:9.7 -8:2:44.0
49.40971379943399 3166.5217376814803
          2015 GH56      13:23:34.5 -8:3:11.0
143.40642085703226 3693.5553838213887
          2015 GT56      13:25:37.1 -8:10:44.0
1853.422842298071 2116.286411329693
          2015 GE56      13:19:4.8 -8:18:54.0
1505.3214742150742 3366.758297968972
          2015 GF56      13:20:23.4 -8:15:7.0
631.5795564837335 3557.3052416038863
          2015 GU55      13:21:54.3 -8:19:41.0
1657.6255442150998 177.63778330369078
          2015 GS56      13:24:22.7 -8:14:18.0
418.65974687961426 33.30306562342279
          2015 GQ56      13:25:13.0 -8:18:8.0
1290.2721768545157 2873.9607262950785
          2015 GF54      13:22:35.2 -8:24:57.0
601.0796949284679 2486.124256644946
          2015 GO56      13:25:20.7 -8:25:30.0
720.6409017640883 3314.8138897764943
433932 61
/astro/store/epyc/users/smotherh/DECAM_Data_Reduction/warps/300
433936


In [88]:
res_df

,visit_id,name,ra_hour,ra_min,ra_sec,dec_deg,dec_min,dec_sec,v_mag,ra_motion,dec_motion,field,ccd,x_pixel,y_pixel,total_motion
0,433932,2015 GF54,13,22,35.2,-8,24,57,24.6,-2,1,300,35,601.079695,2486.124257,2.236068
1,433932,2015 GU55,13,21,54.3,-8,19,41,23.1,-3,1,300,28,1657.625544,177.637783,3.162278
2,433932,2015 GY53,13,22,9.7,-8,2,44,24.3,-8,5,300,21,49.409714,3166.521738,9.433981
3,433932,2015 GH56,13,23,34.5,-8,3,11,24.3,-2,1,300,22,143.406421,3693.555384,2.236068
4,433932,2015 GS56,13,24,22.7,-8,14,18,24.3,-2,1,300,30,418.659747,33.303066,2.236068
5,433932,2015 GG56,13,20,45.5,-8,3,30,24.1,-2,1,300,20,230.271449,2670.487041,2.236068
6,433932,2015 GF56,13,20,23.4,-8,15,7,24.4,-2,1,300,26,631.579556,3557.305242,2.236068
7,433932,2015 GQ56,13,25,13.0,-8,18,8,23.9,-2,0,300,30,1290.272177,2873.960726,2.000000
8,433932,2015 GO56,13,25,20.7,-8,25,30,24.4,-2,0,300,37,720.640902,3314.813890,2.000000
9,433932,2015 GT56,13,25,37.1,-8,10,44,24.5,-2,0,300,24,1853.422842,2116.286411,2.000000


In [89]:
res_df.query('x_pixel > -1 and y_pixel > -1')

,visit_id,name,ra_hour,ra_min,ra_sec,dec_deg,dec_min,dec_sec,v_mag,ra_motion,dec_motion,field,ccd,x_pixel,y_pixel,total_motion
0,433932,2015 GF54,13,22,35.2,-8,24,57,24.6,-2,1,300,35,601.079695,2486.124257,2.236068
1,433932,2015 GU55,13,21,54.3,-8,19,41,23.1,-3,1,300,28,1657.625544,177.637783,3.162278
2,433932,2015 GY53,13,22,9.7,-8,2,44,24.3,-8,5,300,21,49.409714,3166.521738,9.433981
3,433932,2015 GH56,13,23,34.5,-8,3,11,24.3,-2,1,300,22,143.406421,3693.555384,2.236068
4,433932,2015 GS56,13,24,22.7,-8,14,18,24.3,-2,1,300,30,418.659747,33.303066,2.236068
5,433932,2015 GG56,13,20,45.5,-8,3,30,24.1,-2,1,300,20,230.271449,2670.487041,2.236068
6,433932,2015 GF56,13,20,23.4,-8,15,7,24.4,-2,1,300,26,631.579556,3557.305242,2.236068
7,433932,2015 GQ56,13,25,13.0,-8,18,8,23.9,-2,0,300,30,1290.272177,2873.960726,2.000000
8,433932,2015 GO56,13,25,20.7,-8,25,30,24.4,-2,0,300,37,720.640902,3314.813890,2.000000
9,433932,2015 GT56,13,25,37.1,-8,10,44,24.5,-2,0,300,24,1853.422842,2116.286411,2.000000


In [91]:
res_df.query('ccd == 30')

,visit_id,name,ra_hour,ra_min,ra_sec,dec_deg,dec_min,dec_sec,v_mag,ra_motion,dec_motion,field,ccd,x_pixel,y_pixel,total_motion
4,433932,2015 GS56,13,24,22.7,-8,14,18,24.3,-2,1,300,30,418.659747,33.303066,2.236068
7,433932,2015 GQ56,13,25,13.0,-8,18,8,23.9,-2,0,300,30,1290.272177,2873.960726,2.000000
31,433936,2015 GS56,13,24,22.7,-8,14,18,24.3,-2,1,300,30,418.659747,33.303066,2.236068
34,433936,2015 GQ56,13,25,13.0,-8,18,7,23.9,-2,0,300,30,1286.474235,2873.962892,2.000000
58,433940,2015 GS56,13,24,22.6,-8,14,18,24.3,-2,1,300,30,418.664392,27.659593,2.236068
61,433940,2015 GQ56,13,25,13.0,-8,18,7,23.9,-2,0,300,30,1286.474235,2873.962892,2.000000
85,433945,2015 GS56,13,24,22.6,-8,14,17,24.3,-2,1,300,30,414.869636,27.660629,2.236068
88,433945,2015 GQ56,13,25,12.9,-8,18,7,23.9,-2,0,300,30,1286.475391,2868.309000,2.000000
112,433949,2015 GS56,13,24,22.6,-8,14,17,24.3,-2,1,300,30,414.869636,27.660629,2.236068
115,433949,2015 GQ56,13,25,12.9,-8,18,7,23.9,-2,0,300,30,1286.475391,2868.309000,2.000000


In [92]:
res_df.query('ccd == 26')

,visit_id,name,ra_hour,ra_min,ra_sec,dec_deg,dec_min,dec_sec,v_mag,ra_motion,dec_motion,field,ccd,x_pixel,y_pixel,total_motion
6,433932,2015 GF56,13,20,23.4,-8,15,7,24.4,-2,1,300,26,631.579556,3557.305242,2.236068
33,433936,2015 GF56,13,20,23.4,-8,15,7,24.4,-2,1,300,26,631.579556,3557.305242,2.236068
60,433940,2015 GF56,13,20,23.4,-8,15,7,24.4,-2,1,300,26,631.579556,3557.305242,2.236068
87,433945,2015 GF56,13,20,23.4,-8,15,7,24.4,-2,1,300,26,631.579556,3557.305242,2.236068
114,433949,2015 GF56,13,20,23.3,-8,15,7,24.4,-2,1,300,26,631.595226,3551.661425,2.236068
141,434610,2015 GF56,13,20,19.0,-8,14,41,24.4,-2,1,300,26,533.578880,3308.610692,2.236068
168,434614,2015 GF56,13,20,19.0,-8,14,41,24.4,-2,1,300,26,533.578880,3308.610692,2.236068
195,434618,2015 GF56,13,20,18.9,-8,14,41,24.4,-2,1,300,26,533.594428,3302.965498,2.236068
222,434622,2015 GF56,13,20,18.9,-8,14,41,24.4,-2,1,300,26,533.594428,3302.965498,2.236068
249,434626,2015 GF56,13,20,18.9,-8,14,41,24.4,-2,1,300,26,533.594428,3302.965498,2.236068
